# Part 2: Data Cleansing
Data preparation part 2. 

**note: we run this notebook on kaggle.** 

#### From the [first part](https://github.com/cahyaasrini/bangkit-capstone-0323/blob/main/dataset/Part%201%20-%20Data%20Filtering.ipynb), we obtain the **raw dataset of Human OTC Drug Label.** The problem with the raw Human OTC Drug Label dataset is that there are still many missing values but we need all filled dataset especially in some important attributes for our project. There are many attributes in **a drug label**. A drug label in openFDA have 117 attributes by default and some of them are missing in the raw dataset. The raw dataset also have all label versions of a product so that a record is not a unique product.    

#### The data dictionary of the raw  dataset can be found [here](https://drive.google.com/file/d/1btVvh-WcPM5L-vOvTcdiWea7L_LVq35H/view?usp=sharing). 

#### In this notebook, we will cut some records that don't meet the needs as follows: 


1. [According to the FDA](https://www.fda.gov/drugs/information-consumers-and-patients-drugs/otc-drug-facts-label), a drug label must be presented with the standardized format. Here are standardized attributes in a drug label: 
    * active ingredients
    * purpose
    * indications
    * warnings
    * dosage
    * inactive ingredients
   
   Hence, we only use records that have filled standardized attributes. 
 
2. We need **unique product-label** data. To get a unique product, we take the [**upc**](https://en.wikipedia.org/wiki/Universal_Product_Code). We also take **brand_name**, **generic_name**, and also **manufacturer_name** to identify and give more information. 
   
3. To get the lastest label version of a product, since our raw dataset have all versions of a drug label of each product, we will sort the whole data based on its **version**'s **effective_time** and **drop upc duplicates**. 

##### **The whole result can be found [here](https://drive.google.com/drive/folders/1NuOK6hWEDek11kFARszu9K9O8icySx_I?usp=sharing). We also publish the clean-csv version on kaggle [here](https://www.kaggle.com/cahyaasrini/openfda-human-otc-drug-labels).**

In [ ]:
import json

In [ ]:
filename = '../input/openfda-human-otc-drug-labels/fda-otc.json'
with open(filename, 'r') as f:
    data = json.load(f)
    print(len(data))

#### Sample

In [ ]:
# n = 1
# data[str(n)]

#### Number of attribute per record varies

In [ ]:
print(set([len(data[str(i)].keys()) for i in range(1, len(data)+1)]))

#### Make a dataframe and csv file 

In [ ]:
import pandas as pd

In [ ]:
temp = {}

attrs = ['id', 'version', 'effective_time', 
         'purpose', 'indications_and_usage', 
        'active_ingredient', 'inactive_ingredient', 
        'dosage_and_administration','warnings'] 

fda_attrs = ['upc', 'product_type', 'brand_name', 'generic_name', 
             'manufacturer_name'] 

all_attrs = attrs + fda_attrs

for i in data.keys():
    temp[i] = {}  
    for attr in all_attrs:
        try: 
            if attr in fda_attrs:
                temp[i][attr] = data[str(i)]['openfda'][attr][0]
            elif attr == 'effective_time': 
                et = data[str(i)]['effective_time']
                temp[i][attr] = et[:4] + '-' + et[4:6] + '-' + et[6:]
            elif attr == 'id' or attr == 'version':
                temp[i][attr] = data[str(i)][attr]
            else: 
                temp[i][attr] = data[str(i)][attr][0]
        except: 
            temp[i][attr] = None

df = pd.DataFrame.from_dict(temp, orient='index')
df.head(3)

In [ ]:
df.info()

### Drop missing values 

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
df.rename(columns={'id': 'label_id', 'version': 'label_version',
                  'effective_time': 'label_effective_time'}, inplace=True)
df.head(3)

In [ ]:
df.info()

### Keep the lastest version (newest effective_time) only 

In [ ]:
df.sort_values('label_effective_time', ascending=False).head(3)

In [ ]:
df = df.sort_values('label_effective_time', ascending=False).reset_index(drop=True)
df.head(3)

### Drop **upc** duplicates

In [ ]:
df.upc.nunique()

In [ ]:
df.drop_duplicates(subset=['upc'], keep='first', ignore_index=True, inplace=True)
df.info()

In [ ]:
df.head()

### Arrange columns order

In [ ]:
df.columns

In [ ]:
col_order = ['upc', 'product_type', 'brand_name', 'generic_name', 'manufacturer_name',
 'label_id', 'label_version', 'label_effective_time', 
 'purpose', 'indications_and_usage', 'active_ingredient', 'inactive_ingredient',
'dosage_and_administration', 'warnings']

df = df[col_order]
df.head()

In [ ]:
df.info()

### Export to csv 

In [ ]:
# df.to_csv('draft-clean-fda-otc.csv', index=False)

#### check output file

In [ ]:
outcsv = './draft-clean-fda-otc.csv'
df = pd.read_csv(outcsv)
df.info()

In [ ]:
df.dropna(inplace=True)
df.info()

In [ ]:
# df.to_csv('final-clean-fda-otc.csv', index=False)

### Export to json

In [ ]:
to_json = df.to_dict('index')

In [ ]:
len(to_json)

In [ ]:
# to_json['1']

In [ ]:
# with open('final-clean-fda-otc.json','w') as outfile: 
#     json.dump(to_json, outfile, indent=4)    